In [112]:
import pandas as pd
import re
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import unicodedata

##function to remove accents
def strip_accents(text):
    text=text.lower()
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text)

## Getting Soup
def scraping(input_team):
    url='https://dondeverlo.com/futbol/equipo/{}'.format(team)
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html,'html.parser')
    return soup

##next_match
def next_match(soup):
    matches=soup.findAll('h2',{"class":'Event__title'})
    match=matches[0].text
    match=match.lstrip().rstrip()
    return match

##team2
def team2(soup,team):
    matches=soup.findAll('h2',{"class":'Event__title'})
    match=matches[0].text
    match=match.lstrip().rstrip()
    
    nm2=re.sub("\s\s+" , " ",match)
    nm2=strip_accents(nm2)
    nm3=nm2.split(" - ",)
    local=nm3[0]
    visitor=nm3[1]
    if local==team:
        return visitor
    else:
        return local

##date_next_match
def date_next_match(soup):
    dates = soup.findAll('div',{"class":'Event__when'})
    date_match = dates[0].text
    date_match = date_match.lstrip().rstrip()
    return date_match

##competition_next_match
def competition_next_match(soup):
    competition = soup.findAll('span',{"class":'Event__championshipText'})
    competition_match = competition[0].text
    competition_match = competition_match.lstrip().rstrip()
    competition=competition_match.split("-")
    return competition[1].lstrip().rstrip()

##matchday_next_match
def matchday_next_match(soup):
    matchday = soup.findAll('div',{"class":'Event__description'})
    matchday_match = matchday[0].text
    matchday_match = matchday_match.lstrip().rstrip()
    return matchday_match

##summary scrapping next match
def printer(soup,team):
    return "El proximo partido de tu equipo es el {} contra {} correspondiente a la {} de {}.".format(date_match,contender,matchday_match,competition_match)

In [113]:
input_team="sevilla"
team=strip_accents(input_team)
soup=scraping(input_team)
match=next_match(soup)
team2=team2(soup,team)
contender=strip_accents(team2)
date_match=date_next_match(soup)
competition_match=competition_next_match(soup)
matchday_match=matchday_next_match(soup)
                        
print(input_team)
print(team)
print(contender)
print(match)

print("Tu equipo: {}".format(team.upper()))
print("Tu rival: {}".format(contender.upper()))
print(printer(soup,team))

sevilla
sevilla
valladolid
Valladolid - Sevilla
Tu equipo: SEVILLA
Tu rival: VALLADOLID
El proximo partido de tu equipo es el domingo 24 de nov. 21:00 contra valladolid correspondiente a la Jornada 14 de LaLiga.


In [114]:
def dfFilter(team,contender):
    df=pd.read_csv("./input/FMEL_Dataset.csv")
    df['localTeam']=df['localTeam'].map(lambda x: x.lower() if isinstance(x,str) else x)
    df['visitorTeam']=df['visitorTeam'].map(lambda x: x.lower() if isinstance(x,str) else x)
    dff=df[(df['timestamp']>=1093644000) & ((df['localTeam']==(team)) | (df['localTeam']==(contender))) & ((df['visitorTeam']==(contender)) | (df['visitorTeam']==(team)))]
    return dff.sort_values('timestamp', ascending=False)

##function to translate if your team wins, loses, or draws for each game
def result (row,team):
    if row['localGoals']==row['visitorGoals']:
        return 1
    if row['localTeam']==team and row['localGoals']>row['visitorGoals']:
        return 3
    if row['visitorTeam']==team and row['visitorGoals']>row['localGoals']:
        return 3
    else:
        return 0

##creating and appending new column
def summary(a,team,team2):
    a = a.value_counts()
    ##sign=a.value_counts()
    ##sign=df_team['result'].value_counts()
    return "{} y {} se han enfrentado {} veces en los últimos 20 años. El {} ha ganado {} partidos, perdido {}, y empatado {}.".format(team.upper(),team2.upper(),a.sum(),team.upper(),a[3],a[0],a[1])

In [118]:
df_filtered=dfFilter(team,contender)
df_filtered["result"] = df_filtered.apply(lambda df_filtered: result(df_filtered,team), axis=1)
Last_Games=df_filtered.head(5)

print(summary(df_filtered['result'],team,contender))
print(Last_Games)

SEVILLA y VALLADOLID se han enfrentado 10 veces en los últimos 20 años. El SEVILLA ha ganado 3 partidos, perdido 3, y empatado 4.
          id   season  division  round   localTeam visitorTeam  localGoals  \
33214  33215  2013-14         1     28     sevilla  valladolid           4   
33024  33025  2013-14         1      9  valladolid     sevilla           2   
32422  32423  2012-13         1     33  valladolid     sevilla           1   
32234  32235  2012-13         1     14     sevilla  valladolid           1   
29879  29880  2009-10         1     32  valladolid     sevilla           2   

       visitorGoals        date   timestamp  result  
33214             1  16/03/2014  1394924400       3  
33024             2  20/10/2013  1382220000       1  
32422             1  28/04/2013  1367100000       1  
32234             2  03/12/2012  1354489200       0  
29879             1  13/04/2010  1271109600       0  
